https://wikidocs.net/50739 의 내용을 복사한 것입니다.

### 라이브러리 import

In [3]:
import re
from lxml import etree
import urllib.request
import zipfile
from nltk.tokenize import word_tokenize, sent_tokenize

### 데이터 다운로드

In [4]:
urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")
# 데이터 다운로드

with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
  target_text = etree.parse(z.open('ted_en-20160408.xml', 'r'))
  parse_text = '\n'.join(target_text.xpath('//content/text()'))

### 데이터 확인

In [5]:
parse_text[:300]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.\nConsider Facit"

In [9]:
content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text=sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

### 전처리 수행 후의 샘플 갯수

In [10]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [11]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### Word2Vec 훈련시키기

여기서 Word2Vec의 하이퍼파라미터값은 다음과 같습니다.
size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
window = 컨텍스트 윈도우 크기
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
workers = 학습을 위한 프로세스 수
sg = 0은 CBOW, 1은 Skip-gram.


In [14]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [15]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8548706769943237), ('guy', 0.7976933717727661), ('lady', 0.7813059687614441), ('boy', 0.7599497437477112), ('girl', 0.7445526123046875), ('gentleman', 0.7284677028656006), ('soldier', 0.7279588580131531), ('kid', 0.6834539771080017), ('poet', 0.6708505153656006), ('friend', 0.669189453125)]


### 모델의 저장과 로드

In [16]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [17]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8548706769943237), ('guy', 0.7976933717727661), ('lady', 0.7813059687614441), ('boy', 0.7599497437477112), ('girl', 0.7445526123046875), ('gentleman', 0.7284677028656006), ('soldier', 0.7279588580131531), ('kid', 0.6834539771080017), ('poet', 0.6708505153656006), ('friend', 0.669189453125)]


In [22]:
model_result = loaded_model.most_similar("go")
print(model_result)

[('move', 0.7531630992889404), ('walk', 0.7481656074523926), ('come', 0.7122994661331177), ('run', 0.7084965705871582), ('reach', 0.6948980093002319), ('jump', 0.6919817924499512), ('get', 0.6915876865386963), ('take', 0.6883209943771362), ('turn', 0.6879121661186218), ('pass', 0.6804919838905334)]
